<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/langchain-notebooks/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai duckpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00


In [2]:
import os, openai
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [13]:
client = openai.OpenAI()

content = """
다음 <문장>에서 색깔 정보가 있으면 그것을 추출해 영어로 번역하고 <json 포맷>으로 만들어서 출력해라.

<문장>
나는 빨간 사과가 좋더라.
</문장>
"""

In [9]:
def llm_call(content):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": content
            }
        ]
    )
    return response

In [15]:
response = llm_call(content)

print(response.choices[0].message.content)

{
  "색깔": "빨간",
  "영어 번역": "red"
}


In [16]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "color": {
            "type": "string",
            "description": "색깔 정보"
        }
    },
    "required": ["color"]
}

content = f"""
다음 <문장>에서 색깔 정보가 있으면 그것을 추출해 영어로 번역하고 json 포맷으로 만들어서 출력해라.

<문장>
나는 빨간 사과가 좋더라.
</문장>

json schema는 다음과 같다 : {schema}
색깔 정보가 없으면, no color 로 대신해라.
"""

In [17]:
response = llm_call(content)

print(response.choices[0].message.content)

{
    "color": "red"
}


# function

In [21]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": "나는 노란색이 좋더라"
        }],
    functions=[
        {
            "name": "get_color",
            "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력함",
            "parameters": {
                "type": "object",
                "properties": {
                    "color": {
                        "type": "string",
                        "description": "사용자의 선호 색깔을 빼내와서 그에 해당하는 영어 단어를 출력함"
                    }
                },
                "required": ["color"]
            },
        }
    ]
)

In [22]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "color": "노란색"\n}', name='get_color'), tool_calls=None)


In [24]:
print(response.choices[0].message.function_call.name)

get_color


In [23]:
print(response.choices[0].message.function_call.arguments)

{
  "color": "노란색"
}


In [25]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": "아침에 세 개 저녁에 네 개"
        }],
    functions=[
        {
            "name": "apple_add",
            "description": "사과 숫자를 더함",
            "parameters": {
                "type": "object",
                "properties": {
                    "quantity1": {
                        "type": "integer",
                        "description": "첫 번째 숫자"
                    },
                    "quantity2": {
                        "type": "integer",
                        "description": "두 번째 숫자"
                    }

                },
                "required": ["quantity1", "quantity2"]
            },
        }
    ],
    function_call="auto"
)

In [26]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "quantity1": 3,\n  "quantity2": 4\n}', name='apple_add'), tool_calls=None)


# custom function

In [28]:
import openai
import json
from duckpy import Client
import ast

duckduckgo_client = Client()

def duck_search(query) -> str:
    print(f"""
        ## Runs duckduckgo search for query:
        {query}
          """)
    results = duckduckgo_client.search(query)
    return str(results)

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "system",
            "content": "You must use duck_search function to get information"
        },
        {
            "role": "user",
            "content": "원피스는 언제부터 연재했어?"
        }],
    functions=[
        {
            "name": "duck_search",
            "description": "Use to search online",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Translate the Korean content into English input query"
                    },
                },
                "required": ["query"]
            },
        }
    ],
    function_call="auto"
)

In [33]:
print(response)

ChatCompletion(id='chatcmpl-9qjSNvWowrmfzgXBw2blfqQJgvrPr', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "query": "When did One Piece start serializing?"\n}', name='duck_search'), tool_calls=None))], created=1722354819, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=82, total_tokens=104))


In [34]:
message = response.choices[0].message

if message.function_call:
    function_name = message.function_call.name
    args = json.loads(message.function_call.arguments)

    if function_name == "duck_search":
        query = args.get("query")
        results = duck_search(query)
        print(results)

    completion_final = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": "원피스는 언제부터 연재했어?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": results,
            },
        ]
    )
    print(completion_final.choices[0].message.content)


        ## Runs duckduckgo search for query:
        When did One Piece start serializing?
          
[{'title': 'Real-Life Timeline | One Piece Wiki | Fandom', 'description': 'January 1: Eiichiro Oda is born in Kumamoto, Japan. November 20: Eiichiro Oda\'s favorite manga, "Dragon Ball", created by Akira Toriyama, was serialized in Shonen Jump. "Dragon Ball" became an influential hit for Oda to create One Piece 13 years later.[1] At the age of 15 Eiichiro Oda writes his first manga shot story "Fly Up Boy" which scored 18 out of 30 in Shonen Jump.[2] At the age of 17 ...', 'url': 'm/l/?uddg=https://onepiece.fandom.com/wiki/Real-Life_Timeline&rut=76836563723052203de7bc7b69fcf8243ee5962395fcce153fbbbd6441d13306'}, {'title': 'One Piece - Wikipedia', 'description': "One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda.It has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since July 1997, with its individual chapters com

# different output

In [39]:
example_user_input = "런던 날씨가 안 좋다면서?"

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": example_user_input
        }],
    functions=[
        {
            "name": "get_different_responses",
            "description": "답변을 여러가지 말투로 하기",
            "parameters": {
                "type": "object",
                "properties": {
                    "response_up": {
                        "type": "string",
                        "description": "답변을 높임말로 바꿈"
                    },
                    "response_down": {
                        "type": "string",
                        "description": "답변을 반말로 바꿈"
                    }
                },
                "required": ["response_up", "response_down"]
            },
        }
    ],
    # function_call="auto",
    function_call={"name":"get_different_responses"},
)

In [36]:
# function_call = "auto"
print(response.choices[0].message)

ChatCompletionMessage(content='네, 그렇습니다. 런던의 날씨는 현재 좋지 않습니다. 비가 오고 있어요. 우산을 챙기시는 것이 좋을 것 같습니다.', role='assistant', function_call=None, tool_calls=None)


In [40]:
# function_call={"name":"get_different_responses"},
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "response_up": "네, 그렇습니다. 런던의 날씨가 좋지 않다고 전해져왔습니다.",\n  "response_down": "응, 그래. 런던 날씨가 좋지 않대."\n}', name='get_different_responses'), tool_calls=None)


In [41]:
print(response.choices[0].message.function_call.arguments)

{
  "response_up": "네, 그렇습니다. 런던의 날씨가 좋지 않다고 전해져왔습니다.",
  "response_down": "응, 그래. 런던 날씨가 좋지 않대."
}


호출 시점과 별도로 정의한 function A을 사용하고 싶다면
- auto로 선택
- 첫 번째 추론의 결과로 A와 argument가 도착
- 그 때 A를 호출
- A function output과 원래 질문을 두 번째 추론에 던짐

호출 시점에 정의한 function B를 사용하고 싶다면, function_call에 명시

In [44]:
example_user_input = "주말에 친구들 오는데 뭐 먹지? 소고기, 감자, 당근 있어."

def get_recipes_with_ingreients(ingredients:str) -> str:
  return f"{ingredients} 를 다 볶아서 먹읍시다."

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": example_user_input
        }],
    functions=[
        {
            "name": "get_recipes_with_ingreients",
            "description": "주어진 텍스트에서 음식을 만들 수 있는 재료 이름을 뽑아낼 수 있다면 이 function을 사용한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "ingredients": {
                        "type": "string",
                        "description": "음식재료"
                    }
                },
                "required": ["ingredients"]
            },
        }
    ],
    # function_call={"name":"get_recipes_with_ingreients"},
    function_call="auto",
)

In [45]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "ingredients": "소고기, 감자, 당근"\n}', name='get_recipes_with_ingreients'), tool_calls=None)


In [47]:
message = response.choices[0].message

if message.function_call:
    function_name = message.function_call.name
    args = json.loads(message.function_call.arguments)

    if function_name == "get_recipes_with_ingreients":
        ingredients = args.get("ingredients")
        results = get_recipes_with_ingreients(ingredients)
        print(results)

    completion_final = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": example_user_input},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": results,
            },
        ]
    )
    print(completion_final.choices[0].message.content)

소고기, 감자, 당근 를 다 볶아서 먹읍시다.
친구분들이 오시는데, 추가로 약간의 양념재료가 있다면 소고기로 된장찌개나 스튜를 만들어 보는 것은 어떨까요? 감자와 당근을 넣어서 비타민도 함께 섭취할 수 있습니다. 또한, 소고기와 야채를 함께 볶아서 소고기볶음을 만드는 것도 좋습니다. 간단한 각종 양념과 함께 볶으면 단번에 맛있는 요리가 완성됩니다.
